# План домашней работы
1. Реализовать алгоритмы **item2item**, **ALS**, **IALS** (2 балл за каждый)
2. Посчитать метрику предсказаний **MRR@100** выбрасывая случайный лайк пользователя (2 балла)

Будем решать задачу предсказания: на 4/5 пользователей учимся, на 1/5 выбрасываем случайный лайк и пытаемся предсказать его беря топ 100 наших лучших предсказаний для этого пользователя.

MRR@100 будет равно $1/(p+1)$, где $p$ - позиция на которой оказался выброшенный лайк в нашем ранжировании и 0 если в топ 100 его не было.

3. Подобрать параметры алгоритмов для максимизации MRR@100 (1 балл)
4. Сравнить похожести айтемов получаюшиеся для item2item, ALS, IALS (1 балл)

Замерить насколько получаются похожими топы похожестей. Так же рекомендуется взять 5-топовых (или любимых) треков и посмотреть на похожести которые получаются для них в разных алгоритмах.

### Ссылки на датасет музыки:

1. Матричка: https://disk.yandex.ru/d/cLbDbw3mCido_w
2. Имена тайтемов: https://disk.yandex.ru/d/pPCaGJOqcpcABw
3. Линки между айтемами: https://disk.yandex.ru/d/hMErnDJqtVm9HQ

# Разбираемся с датасетом музыки

Взял 0.01 юзеров и 0.005 айтемов от исходного датасета, затем выбросил юзеров, у которых меньше 10 лайков

In [1]:
import os, zipfile
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm.notebook import tqdm, tnrange, trange

from scipy.sparse.linalg import spsolve

In [2]:
# base_name = "yt___home_music_recom_burlada_recsys_2022_02_19_"
# for fname in tqdm([n for n in os.listdir(".") if base_name in n and n.endswith(".zip")]):
#     extract_name = fname.replace(".zip", "")
#     if os.path.exists(extract_name): continue
#     with zipfile.ZipFile(fname) as f:
#         f.extract(f.filelist[0])

In [106]:
# all_names = pd.read_json("yt___home_music_recom_burlada_recsys_2022_02_19_public_all_names", lines=True)
# track_links = pd.read_json("yt___home_music_recom_burlada_recsys_2022_02_19_public_tracks_links", lines=True)

# dataset = pd.read_json("yt___home_music_recom_burlada_recsys_2022_02_19_public_data_train part_2", lines=True)

In [4]:
# dataset

In [5]:
# total_interactions_count = dataset.track_ids.map(len).sum()
# user_coo, item_coo = np.zeros(total_interactions_count, dtype=np.int32), np.zeros(total_interactions_count, dtype=np.int32)
# pos = 0
# for u, tracks in enumerate(tqdm(dataset.track_ids)):
#     user_coo[pos:pos+len(tracks)] = u
#     item_coo[pos:pos+len(tracks)] = tracks
#     pos += len(tracks)
# shape = (max(user_coo) + 1, max(item_coo) + 1)
# user_item_matrix = sp.coo_matrix((np.ones(len(user_coo)), (user_coo, item_coo)), shape=shape)
# user_item_matrix = user_item_matrix.tocsr()
# sp.save_npz("data_train.npz", user_item_matrix)
# # Cleanup memory. Later you need just data_train.npz
# del user_coo
# del item_coo
# del dataset

In [6]:
# from scipy.sparse import vstack

# user_item_matrix1 = sp.load_npz("data_train part_1.npz")
# user_item_matrix2 = sp.load_npz("data_train part_2.npz")
# user_item_matrix = vstack((user_item_matrix1, user_item_matrix2))
# sp.save_npz("data_train.npz", user_item_matrix)
# user_item_matrix.shape

In [93]:
# user_item_matrix = sp.load_npz("data_train.npz")

In [95]:
# item_weights = np.array(user_item_matrix.tocsc().sum(0))[0]
# top_to_bottom_order = np.argsort(-item_weights)
# item_mapping = np.empty(top_to_bottom_order.shape, dtype=int)
# item_mapping[top_to_bottom_order] = np.arange(len(top_to_bottom_order))
# total_item_count = (item_weights > 0).sum()
# total_user_count = user_item_matrix.shape[0]

# def build_dataset(user_item_matrix, item_pct, user_pct):
#     user_count, item_count = int(total_user_count * user_pct), int(total_item_count * item_pct)
#     item_ids = top_to_bottom_order[:item_count]
#     user_ids = np.random.choice(np.arange(user_item_matrix.shape[0]), size=user_count, replace=False)
#     train = user_item_matrix[user_ids]
#     train = train[:, item_ids]
#     return train

In [96]:
# small_dataset = build_dataset(user_item_matrix, 0.005, 0.010)

In [97]:
# sp.save_npz("data_train_not_very_small.npz", small_dataset)

In [98]:
# dataset = sp.load_npz("data_train_very_small.npz").astype(np.int8)
dataset = sp.load_npz("data_train_not_very_small.npz").astype(np.int32)
# dataset = sp.load_npz("data_train_small.npz").astype(np.int8)


print(dataset.__repr__())

dataset = dataset.toarray()
dataset = dataset[dataset.sum(axis=1) >= 10]
dataset = sp.csr_matrix(dataset)

print(dataset.__repr__())

<13732x5000 sparse matrix of type '<class 'numpy.int32'>'
	with 742963 stored elements in Compressed Sparse Row format>
<9916x5000 sparse matrix of type '<class 'numpy.int32'>'
	with 726459 stored elements in Compressed Sparse Row format>


# 1. Реализация алгоритмов

## item2item

Порога по количеству пересечений у айтемов не делал, соседей отсекал не по значению похожести, а по количеству.

In [19]:
def pairwise_jaccard(X):
    """
    Computes the Jaccard distance between the rows of `X`.
    source: https://stackoverflow.com/questions/32805916/compute-jaccard-distances-on-sparse-matrix
    """
    X = X.astype(bool).astype(int)

    intrsct = X.dot(X.T)
    row_sums = intrsct.diagonal()
    unions = row_sums[:, None] + row_sums - intrsct
    sim = intrsct / unions
    # dist = 1 - sim
    return np.array(sim)

def compute_neighbors(similarities, num_neighbors=1000):
    argsorted_similarities = np.argsort(similarities, axis=1)[:, ::-1]
    return argsorted_similarities[:, :num_neighbors]

def compute_r_hat(dataset, similarities, items_neighbors):
    ratings = dataset.toarray()
    ratings_hat = np.zeros_like(ratings)
    
    for i in trange(ratings.shape[1], leave=False):
        Ni = items_neighbors[i]
        R = ratings[:, Ni]
        s = similarities[i, Ni]
        
        ratings_hat[:, i] = R @ s / np.sum(np.abs(s))
            
    return ratings_hat

# similarities = pairwise_jaccard(dataset.T)
# items_neighbors = compute_neighbors(similarities, 100 // 2)
# items_neighbors.shape
# ratings_hat = compute_r_hat(dataset, similarities, items_neighbors)

In [20]:
class Item2Item:    
    def __init__(self, neighbors_size=50):
        self.neighbors_size = neighbors_size
    
    def fit(self, dataset):
        self.dataset = dataset
        self.ratings = dataset.toarray()
        
        self.similarities = pairwise_jaccard(self.dataset.T)
        self.items_neighbors = compute_neighbors(self.similarities, self.neighbors_size)
        self.ratings_hat = compute_r_hat(self.dataset, self.similarities, self.items_neighbors)
        
    def get_top_for_user(self, u, size=100):
        return np.argsort(self.ratings_hat[u, self.ratings[u, :] != 1])[::-1][:size]

In [32]:
i2i = Item2Item()
i2i.fit(dataset)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [33]:
i2i.get_top_for_user(0, 100)

array([4946, 1650, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1651, 1660,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1642, 1641, 1640, 1639,
       1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634,
       1635, 1636, 1637, 1638, 1659, 1661, 1699, 1689, 1682, 1683, 1684,
       1685, 1686, 1687, 1688, 1690, 1662, 1691, 1692, 1693, 1694, 1695,
       1696, 1697, 1681, 1680, 1679, 1678, 1663, 1664, 1665, 1666, 1667,
       1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1623,
       1622, 1621, 1573, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1574,
       1620, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1565, 1564, 1563,
       1562])

## ALS

Источники:  
http://activisiongamescience.github.io/2016/01/11/Implicit-Recommender-Systems-Biased-Matrix-Factorization/#Alternating-Least-Squares  
https://academy.yandex.ru/handbook/ml/article/rekomendacii-na-osnove-matrichnyh-razlozhenij

In [8]:
def add_some_random_dislikes(dataset, max_dislikes=100):
    e_dataset = dataset.copy()
    for u in trange(dataset.shape[0]):
        row = dataset[u, :].toarray()
        likes_count = (row == 1).sum()
        
        dislikes_ind = np.random.choice(np.where(row == 0)[0], size=min(likes_count, max_dislikes), replace=False)
        for i in dislikes_ind:
            e_dataset[u, i] = -1
        
    return e_dataset

e_dataset = add_some_random_dislikes(dataset, 100)
e_dataset

  0%|          | 0/9916 [00:00<?, ?it/s]

/home/zalevskiaa/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


<9916x5000 sparse matrix of type '<class 'numpy.int32'>'
	with 735247 stored elements in Compressed Sparse Row format>

In [232]:
def explicit_als(dataset, iterations=10, lambda_val=0.1, S=10):    
    N, D = dataset.shape
    t_dataset = dataset.T
    
    X = sp.csr_matrix(np.random.normal(size = (N, S)))
    Y = sp.csr_matrix(np.random.normal(size = (D, S)))
    
    I = sp.eye(S)
    lI = lambda_val * I
    
    for it in trange(iterations, leave=False):        
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        for u in trange(N, leave=False):
            ru = dataset[u].toarray().reshape(-1)
            # print(Y.T.shape, ru.shape)
            yT_ru = Y.T.dot(ru)
            X[u] = spsolve(yTy + lI, yT_ru)
            
        for i in trange(D, leave=False):
            ri = t_dataset[i].toarray().reshape(-1)
            # print(X.T.shape, ri.shape)
            xT_ri = X.T.dot(ri)
            Y[i] = spsolve(xTx + lI, xT_ri)
        
    return X, Y

In [10]:
class ALS:
    def __init__(self, iterations=4, lambda_val=0.1, features=20):
        self.iterations = iterations
        self.lambda_val = lambda_val
        self.features = features
        
    def fit(self, dataset):
        self.X, self.Y = explicit_als(dataset, self.iterations, self.lambda_val, self.features)
        self.ratings = dataset.toarray()
        self.ratings_hat = (self.X @ self.Y.T).toarray()
        
    def get_top_for_user(self, u, size=100):
        return np.argsort(self.ratings_hat[u, self.ratings[u, :] != 1])[::-1][:size]

In [77]:
als = ALS(iterations=1)
als.fit(dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [78]:
als.get_top_for_user(0)

array([ 158,   75,  132,  202,  443,   20,   43,  186,   83,   50,  142,
         32,    2,    8,  628,  618,   35,   64,   51,  294,  575,   66,
         70,  115,   36,  336,  271,  201,  174,  257,  234,  544,   98,
        568,  624,  235,   76,  421,  610,   18,  477,   46,  716,  341,
        258, 1337, 1627,  259,  473,  129,  845,  308,   30,  133,  480,
        989,  564,  507, 1333,  130,  654,  906,    3, 2274,   34,  581,
       1287,   60,  597,  493, 1104, 1180,  673, 2470, 1332,  714,  276,
       1776,  231,  537,  571,  250,  233,  192, 2348, 2567, 1341, 2065,
          6,  854,  217,   42,  210,  635,  371,  119,  232,   86,  578,
        272])

In [12]:
# als.ratings_hat

## IALS

Источники:  
http://activisiongamescience.github.io/2016/01/11/Implicit-Recommender-Systems-Biased-Matrix-Factorization/#Implicit-Feedback  
https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe  
https://academy.yandex.ru/handbook/ml/article/rekomendacii-na-osnove-matrichnyh-razlozhenij

In [21]:
def compute_f(dataset, X, Y, confidence, lambda_val):
    diff = dataset - X.dot(Y.T)
    
    # print(confidence.shape, diff.shape)
    
    f = (confidence.toarray() * diff.toarray() ** 2).sum()
    f += lambda_val * (np.linalg.norm(X.toarray(), axis=1) ** 2).sum()
    f += lambda_val * (np.linalg.norm(Y.toarray(), axis=0) ** 2).sum()
    return f

def implicit_als(dataset, alpha=40, iterations=10, lambda_val=0.1, S=10):  
    """
    source : https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe
    """
    
    confidence = dataset * alpha
    N, D = dataset.shape
    
    X = sp.csr_matrix(np.random.normal(size = (N, S)))
    Y = sp.csr_matrix(np.random.normal(size = (D, S)))
    
    X_I = sp.eye(N)
    Y_I = sp.eye(D)
    
    I = sp.eye(S)
    lI = lambda_val * I
    
    for it in trange(iterations, leave=False):        
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        for u in trange(N, leave=False):
            u_row = confidence[u, :].toarray() 

            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0

            CuI = sp.diags(u_row, [0])
            Cu = CuI + Y_I

            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

        for i in trange(D, leave=False):
            i_row = confidence[:, i].T.toarray()

            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            CiI = sp.diags(i_row, [0])
            Ci = CiI + X_I

            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)
        
        # f = compute_f(dataset, X, Y, confidence, lambda_val)
        # print(f)
        
    return X, Y

In [22]:
class IALS:
    def __init__(self, alpha=40, iterations=4, lambda_val=0.1, features=20):
        self.alpha = alpha
        self.iterations = iterations
        self.lambda_val = lambda_val
        self.features = features
        
    def fit(self, dataset):
        self.X, self.Y = implicit_als(dataset, self.alpha, self.iterations, self.lambda_val, self.features)
        self.ratings = dataset.toarray()
        self.ratings_hat = (self.X @ self.Y.T).toarray()
        
    def get_top_for_user(self, u, size=100):
        return np.argsort(self.ratings_hat[u, self.ratings[u, :] != 1])[::-1][:size]

In [79]:
ials = IALS(iterations=1)
ials.fit(dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [80]:
ials.get_top_for_user(0)

array([ 15,   7,  10, 101,  32,  54,  41,  20,  13,   8,  29,  50,  95,
         1,  17,  19,  11,  53, 149,  72,   4,  52, 111,  67,  38, 118,
        82,  49,  45, 114,  48, 157,  62,  76, 127,  89,  80, 116,  33,
        93, 237,  66,  44, 148, 193, 190, 138, 163, 274, 112, 139,  90,
        39, 196, 151, 185,  79, 170, 150,  73, 173, 200,  34, 168, 134,
       368, 357, 204,  12, 129, 342, 100, 205, 132, 445,  42, 456, 289,
       249, 345, 321, 229, 104, 102,  96, 405, 135, 427, 123,  22,  31,
       156,  84, 142, 337,  35, 136, 359, 275, 221])

# 2. MRR@100

Делал так:  
- Модифицируется датасет: для выбранных (1/5) пользователей убирается по отдому случайному лайку
- На модифицированном датасете обучается модель, для выбранных пользователей рекомендуется топ 100 айтемов
- Собираются позиции выбранных случайных лайков в топах-100, считается среднее значение 1 / (1 + p) 

mrr-ы получились на порядок ниже, чем у ребят в чате, мб потому что я по-другому их считал, либо недостаточно потюнил параметры. Не бейте сильно пожалуйста

In [23]:
def choose_random_items(ratings, test_users):
    chosen_items = []
    for row in ratings[test_users, :]:
        likes_indices = np.where(row == 1)[0]
        if likes_indices.shape[0] > 0:
            chosen_items.append(np.random.choice(likes_indices))
        else:
            chosen_items.append(-1)
    return np.array(chosen_items)


def remove_chosen_items(ratings, test_users, chosen_items):
    modified_ratings = ratings.copy()
    
    for u, i in zip(test_users, chosen_items):
        if i == -1:
            continue
            
        assert modified_ratings[u, i] == 1
        modified_ratings[u, i] = 0
    
    return modified_ratings


def compute_mrr(dataset, model):
    # print(type(dataset))
    
    ratings = dataset.toarray()
    N, D = ratings.shape
    
    test_users = np.arange(N - N // 5, N)
    chosen_items = choose_random_items(ratings, test_users)
    # print(chosen_items.dtype)
    
    modified_ratings = remove_chosen_items(ratings, test_users, chosen_items) # to sparse
    modified_dataset = sp.csr_matrix(modified_ratings)
    # print(modified_dataset.dtype)
    
    model.fit(modified_dataset)
    # model.fit(dataset)
    
    mrrs = []
    for u, i in tqdm(zip(test_users, chosen_items), total=test_users.shape[0], leave=False):
        top = model.get_top_for_user(u)
        
        pos = np.where(top == i)[0] # array
        mrr = 1 / (pos[0] + 1) if pos.shape[0] > 0 else 0
        mrrs.append(mrr)
    mrrs = np.array(mrrs)    
    
    return mrrs.mean()

In [24]:
compute_mrr(dataset, Item2Item(neighbors_size=100))

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/1983 [00:00<?, ?it/s]

0.0010326685836356709

In [25]:
compute_mrr(e_dataset, ALS(iterations=1))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/1983 [00:00<?, ?it/s]

0.005099959289752171

In [26]:
compute_mrr(dataset, IALS(iterations=1))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/1983 [00:00<?, ?it/s]

0.0062626160237228305

# 3. Максимизация MRR@100

In [76]:
def run_i2i():
    for ns in [10, 30, 50, 100, 200, 500, 1000]:
        mrr = compute_mrr(dataset, Item2Item(neighbors_size=ns))
        print(ns, mrr)

def run_als():
    for S in [10, 20, 30]:
        for lam in [0.001, 0.1, 1]:
            mrr = compute_mrr(e_dataset, ALS(iterations=6, lambda_val=lam, features=S))
            print(S, lam, mrr)
            
def run_ials():
    for S in [20, 30, 40]:
        for lam in [0.1, 1]:
            mrr = compute_mrr(dataset, IALS(iterations=6, lambda_val=lam, features=S))
            print(S, lam, mrr)

In [77]:
# run_i2i()

In [78]:
# run_als()

In [79]:
# run_ials()

Перебирал параметры. Получил вот такие результаты:

In [30]:
import pandas as pd

print('item-2-item')
pd.DataFrame({'neighbors_size': [10, 30, 50, 100, 200, 500, 1000], 'MRR@100': [0.00077, 0.000534, 0.00086, 0.00102, 0.00159, 0.00138, 0.00067]})

item-2-item


,neighbors_size,MRR@100
0,10,0.000770
1,30,0.000534
2,50,0.000860
3,100,0.001020
4,200,0.001590
5,500,0.001380
6,1000,0.000670


In [27]:
print('ALS')
pd.DataFrame({
    'features': [10, 10, 10, 20, 20, 20, 30, 30, 30], 
    'lambda': [0.01, 0.1, 1, 0.01, 0.1, 1, 0.01, 0.1, 1], 
    'MRR@100': [0.0046, 0.0048, 0.0068, 0.0053, 0.00705, 0.0095, 0.0044, 0.0039, 0.009]})

ALS


,features,lambda,MRR@100
0,10,0.01,0.00460
1,10,0.10,0.00480
2,10,1.00,0.00680
3,20,0.01,0.00530
4,20,0.10,0.00705
5,20,1.00,0.00950
6,30,0.01,0.00440
7,30,0.10,0.00390
8,30,1.00,0.00900


In [28]:
print('IALS')
pd.DataFrame({
    'features': [20, 20, 30, 30, 40, 40], 
    'lambda': [0.1, 1, 0.1, 1, 0.1, 1], 
    'MRR@100': [0.0052, 0.0057, 0.0069, 0.0051, 0.0059, 0.0053]})

IALS


,features,lambda,MRR@100
0,20,0.1,0.0052
1,20,1.0,0.0057
2,30,0.1,0.0069
3,30,1.0,0.0051
4,40,0.1,0.0059
5,40,1.0,0.0053


In [29]:
print('Best parameters')
pd.DataFrame({
    'model': ['item-2-item', 'ALS', 'IALS'],
    'MRR@100': [0.001590, 0.00950, 0.0069],
    'neighbors_size': [200, pd.NA, pd.NA], 
    'features': [pd.NA, 20, 30],
    'lambda': [pd.NA, 1, 0.1], 
}).sort_values('MRR@100', ascending=False)

Best parameters


,model,MRR@100,neighbors_size,features,lambda
1,ALS,0.00950,<NA>,20,1
2,IALS,0.00690,<NA>,30,0.1
0,item-2-item,0.00159,200,<NA>,<NA>


# 4. Сравнение похожестей айтемов для item2item, ALS, IALS

Сравним топы похожих для одного айтема у разных моделей - в каждой паре топов посчитаем количество общих айтемов

In [38]:
i2i = Item2Item(neighbors_size=200)
i2i.fit(dataset)

als = ALS(features=20, lambda_val=1)
als.fit(e_dataset)

ials = IALS(features=30, lambda_val=0.1)
ials.fit(dataset)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/9916 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [219]:
def compare_similar_items_tops(item, top_size=100):
    i2i_neighbors = i2i.items_neighbors[item][:top_size]
    # i2i.similarities[item, i2i.items_neighbors[item]][:top_size]
    
    als_similarities = als.Y.dot(als.Y.T)[item, :].toarray().reshape(-1)
    als_neighbors = np.argsort(als_similarities)[::-1][:top_size]
    
    ials_similarities = ials.Y.dot(ials.Y.T)[item, :].toarray().reshape(-1)
    ials_neighbors = np.argsort(ials_similarities)[::-1][:top_size]
    
    print('item', item, 'top_size', top_size)
    print('i2i - als ', np.intersect1d(i2i_neighbors, als_neighbors).size)
    print('i2i - ials ', np.intersect1d(i2i_neighbors, ials_neighbors).size)
    print('als - ials ', np.intersect1d(als_neighbors, ials_neighbors).size)
    print()

In [217]:
compare_similar_items_tops(0, 200)
compare_similar_items_tops(705, 200)
compare_similar_items_tops(2001, 200)
compare_similar_items_tops(3006, 200)
compare_similar_items_tops(1984, 200)

item 0 top_size 200
i2i - als  134
i2i - ials  128
als - ials  167

item 705 top_size 200
i2i - als  61
i2i - ials  56
als - ials  158

item 2001 top_size 200
i2i - als  6
i2i - ials  4
als - ials  167

item 3006 top_size 200
i2i - als  5
i2i - ials  6
als - ials  147

item 1984 top_size 200
i2i - als  13
i2i - ials  13
als - ials  167



In [218]:
compare_similar_items_tops(0, 500)
compare_similar_items_tops(705, 500)
compare_similar_items_tops(2001, 500)
compare_similar_items_tops(3006, 500)
compare_similar_items_tops(1984, 500)

item 0 top_size 500
i2i - als  189
i2i - ials  187
als - ials  424

item 705 top_size 500
i2i - als  100
i2i - ials  92
als - ials  394

item 2001 top_size 500
i2i - als  12
i2i - ials  16
als - ials  395

item 3006 top_size 500
i2i - als  32
i2i - ials  28
als - ials  387

item 1984 top_size 500
i2i - als  35
i2i - ials  29
als - ials  410



Топы треков Item2Item плохо пересекаются с топами ALS и IALS, но лучше, если исходный трек популярный

**Получим списки названий похожих треков**

In [103]:
# item_weights = np.array(sp.load_npz("data_train.npz").tocsc().sum(0))[0]
# top_to_bottom_order = np.argsort(-item_weights)
# item_mapping = np.empty(top_to_bottom_order.shape, dtype=int)
# item_mapping[top_to_bottom_order] = np.arange(len(top_to_bottom_order))
# total_item_count = (item_weights > 0).sum()

# item_ids = top_to_bottom_order[:int(total_item_count * 0.005)]
# np.save('item_ids_not_very_small', item_ids)

In [104]:
item_ids = np.load('item_ids_not_very_small.npy')

In [109]:
all_names = pd.read_json("yt___home_music_recom_burlada_recsys_2022_02_19_public_all_names", lines=True)
all_names = all_names[all_names.type == 'track']

In [174]:
def get_name_by_item(item):
    return list(all_names[all_names.id == item_ids[item]]['name'])[0]

def item_to_id(item):
    return item_ids[item]

def id_to_item(i):
    res = np.where(item_ids == i)
    if len(res) > 0:
        return res[0][0]
    return None

In [175]:
id_to_item(19079574), item_to_id(1052)

(1052, 19079574)

In [230]:
# popular tracks
for item, i in enumerate(item_ids[:10]):
    print(item, i, get_name_by_item(item))

0 53404 Smells Like Teen Spirit
1 33311009 Believer
2 178529 Numb
3 35505245 I Got Love
4 65851540 Юность
5 24692821 Way Down We Go
6 32947997 Shape of You
7 51241318 In The End
8 795836 Shape Of My Heart
9 45499814 Life


In [231]:
# search by name
all_names[all_names.name.str.contains('Гор') & all_names.id.isin(item_ids)]

,type,id,name
829704,track,106160,Город
939394,track,2286226,Город
959316,track,3616404,Гореть
1095937,track,17926967,"Город, которого нет"
1213737,track,27317888,Город
1222036,track,28130240,Город Сочи
1226543,track,28420997,Город под подошвой
1344696,track,36843979,Город золотой
1356114,track,37572518,Горы по колено
1484538,track,48591946,Город дорог


In [225]:
def write_i2i_similar_names(i2i, item, top_size=10):
    print(f'{item} {get_name_by_item(item)}:')
    for i in i2i.items_neighbors[item][:top_size]:
        print(f'{i:7} {get_name_by_item(i)}')
    print()
        
def write_als_similar_names(als, item, top_size=10):
    als_similarities = als.Y.dot(als.Y.T)[item, :].toarray().reshape(-1)
    als_neighbors = np.argsort(als_similarities)[::-1][:top_size]
    
    print(f'{item} {get_name_by_item(item)}:')
    for i in als_neighbors:
        print(f'{i:7} {get_name_by_item(i)}')
    print()

In [226]:
write_i2i_similar_names(i2i, 0)
write_als_similar_names(als, 0)
write_als_similar_names(ials, 0)

0 Smells Like Teen Spirit:
      0 Smells Like Teen Spirit
      2 Numb
     11 Seven Nation Army
     14 Zombie
     38 Californication
     43 Highway to Hell
    309 Come As You Are
     18 Bring Me To Life
     30 It's My Life
     16 The Show Must Go On

0 Smells Like Teen Spirit:
      0 Smells Like Teen Spirit
      2 Numb
      1 Believer
     18 Bring Me To Life
     11 Seven Nation Army
      5 Way Down We Go
     13 Human
      8 Shape Of My Heart
     16 The Show Must Go On
     12 In My Mind

0 Smells Like Teen Spirit:
      0 Smells Like Teen Spirit
      1 Believer
      2 Numb
     11 Seven Nation Army
     16 The Show Must Go On
      8 Shape Of My Heart
      9 Life
     14 Zombie
      5 Way Down We Go
     18 Bring Me To Life



In [227]:
write_i2i_similar_names(i2i, 1052)
write_als_similar_names(als, 1052)
write_als_similar_names(ials, 1052)

1052 Беспечный ангел:
   1052 Беспечный ангел
    839 Потерянный рай
   1557 Возьми моё сердце
     65 Я свободен
    575 Осколок льда
    585 Звезда по имени Солнце
   1020 Хочу перемен
    382 Группа крови
    518 Штиль
   1613 Кукушка

1052 Беспечный ангел:
      0 Smells Like Teen Spirit
      2 Numb
     18 Bring Me To Life
      1 Believer
     11 Seven Nation Army
     14 Zombie
     16 The Show Must Go On
      8 Shape Of My Heart
      5 Way Down We Go
     51 In the End

1052 Беспечный ангел:
      1 Believer
      2 Numb
      0 Smells Like Teen Spirit
     16 The Show Must Go On
     11 Seven Nation Army
     18 Bring Me To Life
     14 Zombie
      8 Shape Of My Heart
      6 Shape of You
     65 Я свободен



In [228]:
write_i2i_similar_names(i2i, 3423)
write_als_similar_names(als, 3423)
write_als_similar_names(ials, 3423)

3423 Воины света:
   3423 Воины света
   4781 12 обезьян
    948 В руках автомат
    646 Воины света
   1497 Я верю
   2623 Шут
   3589 Гитары
   2597 Россия для грустных
   3339 Северный флот
   1737 Андерграунд

3423 Воины света:
      0 Smells Like Teen Spirit
      2 Numb
     18 Bring Me To Life
      1 Believer
     16 The Show Must Go On
     14 Zombie
     47 Выхода нет
      8 Shape Of My Heart
     51 In the End
      9 Life

3423 Воины света:
      0 Smells Like Teen Spirit
      1 Believer
      2 Numb
      4 Юность
     11 Seven Nation Army
     14 Zombie
      8 Shape Of My Heart
     16 The Show Must Go On
      9 Life
     33 Зелёные волны



In [229]:
write_i2i_similar_names(i2i, 681)
write_als_similar_names(als, 681)
write_als_similar_names(ials, 681)

681 Гореть:
    681 Гореть
    625 Sid & Nancy
    996 Три полоски
    747 Вселенная бесконечна?
    766 Выдыхай
    457 Прости. Прощай. Привет
   2896 Дыши со мной
   1171 Кофе – мой друг
   1437 Три полоски
   3587 На Марсе классно

681 Гореть:
      0 Smells Like Teen Spirit
      2 Numb
      1 Believer
     18 Bring Me To Life
      5 Way Down We Go
     11 Seven Nation Army
     14 Zombie
     13 Human
      8 Shape Of My Heart
     16 The Show Must Go On

681 Гореть:
      2 Numb
      0 Smells Like Teen Spirit
     18 Bring Me To Life
     11 Seven Nation Army
      1 Believer
     14 Zombie
     16 The Show Must Go On
     51 In the End
      8 Shape Of My Heart
     38 Californication



- ALS и IALS чаще, чем Item2Item предлагают популярные треки  
- Ещё есть такое ощущение, что Item2Item предлагает "слишком" похожие треки по сравнению с ALS, IALS, то есть если слушать список, предложенный, i2i, то он быстро надоест, а ALS, IALS скорее предлагают треки, похожие по "стилю/жанру"